# Health Insurance Recommendation System with CrewAI

This notebook uses **CrewAI** with **Ollama (llama3.2)** to provide personalized health insurance recommendations.

## System Architecture
- **User Profiler Agent**: Analyzes user inputs and creates a detailed profile
- **Recommendation Agent**: Matches user profile against all plans and ranks them
- **Comparison Agent**: Creates detailed comparison with reasoning

In [5]:
# Import required libraries
import json
import pandas as pd
from crewai import Agent, Task, Crew, Process
from langchain_community.llms import Ollama
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [6]:
# Load insurance data
with open('data/indian_health_insurance_data.json', 'r') as f:
    insurance_data = json.load(f)

# Extract insurers and plans
insurers = insurance_data['insurers']
total_plans = sum(len(insurer['plans']) for insurer in insurers)

print(f"✅ Loaded data for {len(insurers)} insurers")
print(f"✅ Total plans available: {total_plans}")
print(f"\nInsurers: {', '.join([i['insurer_name'] for i in insurers])}")

✅ Loaded data for 6 insurers
✅ Total plans available: 9

Insurers: Star Health and Allied Insurance Co. Ltd., HDFC ERGO General Insurance Company Ltd., ICICI Lombard General Insurance Co. Ltd., Niva Bupa Health Insurance Co. Ltd., Care Health Insurance Ltd., Aditya Birla Health Insurance Co. Ltd.


In [7]:
# Configure Ollama LLM with llama3.2
llm = Ollama(
    model="llama3.2",
    base_url="http://localhost:11434",
    temperature=0.7
)

print("✅ Ollama LLM configured with llama3.2 model")

✅ Ollama LLM configured with llama3.2 model


In [8]:
# Define CrewAI Agents

# Agent 1: User Profiler
user_profiler = Agent(
    role='Health Insurance Needs Analyst',
    goal='Analyze user requirements and create a comprehensive profile of their insurance needs',
    backstory="""You are an expert health insurance analyst who specializes in understanding 
    customer needs. You excel at identifying the most important factors for each customer based 
    on their age, health conditions, budget, and preferences. You create detailed profiles that 
    help match customers with the right insurance plans.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# Agent 2: Recommendation Agent
recommendation_agent = Agent(
    role='Insurance Plan Matcher',
    goal='Analyze all available insurance plans and rank the top 5 best matches for the user profile',
    backstory="""You are a highly experienced insurance plan matcher with deep knowledge of 
    Indian health insurance products. You excel at analyzing plan features like waiting periods, 
    coverage limits, claim settlement ratios, and special benefits. You can quickly identify 
    which plans best match a customer's unique needs and budget constraints.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# Agent 3: Comparison Agent
comparison_agent = Agent(
    role='Insurance Comparison Specialist',
    goal='Create a detailed comparison of recommended plans with clear reasoning for rankings',
    backstory="""You are an insurance comparison expert who excels at breaking down complex 
    insurance features into simple, understandable comparisons. You provide clear reasoning for 
    why certain plans are better suited for specific customer needs. You focus on practical 
    benefits and highlight key differentiators.""",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

print("✅ Created 3 specialized agents:")
print("   1. User Profiler Agent")
print("   2. Recommendation Agent")
print("   3. Comparison Agent")

Error instantiating LLM from unknown object type: Error importing native provider: OPENAI_API_KEY is required


ImportError: Error importing native provider: OPENAI_API_KEY is required

In [ ]:
# User Input Collection
print("=" * 60)
print("HEALTH INSURANCE RECOMMENDATION SYSTEM")
print("=" * 60)
print("\nPlease provide your details for personalized recommendations:\n")

user_age = input("1. Your Age: ")
user_ped = input("2. Pre-existing Conditions (comma-separated, or 'None'): ")
user_budget = input("3. Budget Range (e.g., '15000-25000' per year): ")
user_needs = input("4. Specific Needs (e.g., maternity, OPD, no room rent limit): ")
user_preferences = input("5. Preferred Features (e.g., wellness rewards, quick claim settlement): ")

# Create user profile dictionary
user_profile = {
    'age': user_age,
    'pre_existing_conditions': user_ped,
    'budget': user_budget,
    'specific_needs': user_needs,
    'preferences': user_preferences
}

print("\n✅ User profile collected successfully!")
print(f"\nProfile Summary:")
for key, value in user_profile.items():
    print(f"  - {key.replace('_', ' ').title()}: {value}")

In [ ]:
# Convert insurance data to a compact context string for agents
def create_insurance_context():
    """Create a concise context string with all insurance plan details"""
    context = "AVAILABLE HEALTH INSURANCE PLANS:\n\n"
    
    for insurer in insurers:
        context += f"\n{'='*60}\n"
        context += f"INSURER: {insurer['insurer_name']}\n"
        context += f"CSR: {insurer['claim_settlement_ratio']['csr_percentage']}\n"
        context += f"Network Hospitals: {insurer['network_hospitals']}\n"
        
        for plan in insurer['plans']:
            context += f"\n--- PLAN: {plan['plan_name']} ---\n"
            context += f"Type: {plan['plan_type']}\n"
            context += f"Description: {plan['description']}\n"
            
            # Coverage details
            coverage = plan['coverage_static']
            context += f"\nCOVERAGE:\n"
            context += f"- Sum Insured: {coverage.get('sum_insured_options', 'N/A')}\n"
            context += f"- Room Rent Limit: {coverage.get('room_rent_limit', 'N/A')}\n"
            context += f"- Co-payment: {coverage.get('co_payment', 'N/A')}\n"
            context += f"- NCB: {coverage.get('no_claim_bonus', 'N/A')}\n"
            context += f"- Restoration: {coverage.get('restoration_benefit', 'N/A')}\n"
            context += f"- Maternity: {coverage.get('maternity_coverage', 'N/A')}\n"
            context += f"- OPD: {coverage.get('opd_coverage', 'N/A')}\n"
            context += f"- Pre-hospitalization: {coverage.get('pre_hospitalization_days', 'N/A')} days\n"
            context += f"- Post-hospitalization: {coverage.get('post_hospitalization_days', 'N/A')} days\n"
            
            # Waiting periods
            waiting = plan['waiting_periods']
            context += f"\nWAITING PERIODS:\n"
            context += f"- Initial: {waiting.get('initial_waiting_days', 'N/A')} days\n"
            context += f"- PED: {waiting.get('ped_waiting_months', 'N/A')} months\n"
            if waiting.get('ped_buyback_available'):
                context += f"- PED Buyback: Available (reduces to {waiting.get('ped_buyback_reduced_to_months', 'N/A')} months)\n"
            context += f"- Specific Diseases: {waiting.get('specific_diseases_months', 'N/A')} months\n"
            
            # Key features
            context += f"\nKEY FEATURES: {', '.join(plan.get('key_features', [])[:5])}\n"
    
    return context

insurance_context = create_insurance_context()
print(f"✅ Created insurance context ({len(insurance_context)} characters)")

In [ ]:
# Define CrewAI Tasks

# Task 1: Profile the user
profile_task = Task(
    description=f"""Analyze the following user information and create a detailed needs profile:
    
    User Information:
    - Age: {user_profile['age']}
    - Pre-existing Conditions: {user_profile['pre_existing_conditions']}
    - Budget: {user_profile['budget']}
    - Specific Needs: {user_profile['specific_needs']}
    - Preferences: {user_profile['preferences']}
    
    Create a comprehensive profile that identifies:
    1. Priority features based on age and health status
    2. Must-have coverage requirements
    3. Budget constraints and flexibility
    4. Key decision factors
    
    Output a concise profile summary.""",
    agent=user_profiler,
    expected_output="A detailed user needs profile with priority requirements"
)

# Task 2: Recommend plans
recommend_task = Task(
    description=f"""Based on the user profile, analyze ALL available insurance plans and 
    recommend the TOP 5 BEST MATCHES.
    
    Available Insurance Plans:
    {insurance_context}
    
    Evaluation Criteria:
    1. Match with user's specific needs
    2. Budget compatibility (prefer plans close to budget; if over budget, mention it)
    3. Waiting period considerations (especially for PED if user has pre-existing conditions)
    4. Claim settlement ratio
    5. Coverage features vs. needs
    
    Output format:
    Rank each plan (1-5) with:
    - Plan Name
    - Insurer Name
    - Match Score (1-10)
    - Brief reason for ranking
    
    IMPORTANT: If the user's budget is lower than typical premiums, still recommend the 5 best 
    value-for-money options and note budget considerations.""",
    agent=recommendation_agent,
    expected_output="Top 5 ranked insurance plans with match scores and reasoning",
    context=[profile_task]
)

# Task 3: Create comparison
compare_task = Task(
    description="""Create a detailed comparison table of the top 5 recommended plans.
    
    For each plan, provide:
    1. Rank (1-5)
    2. Plan Name
    3. Insurer
    4. Key Features (3-4 bullet points)
    5. Why Recommended (2-3 brief bullet points)
    6. Important Considerations (if any, like budget, waiting periods)
    
    Format the output as a structured comparison that can be easily converted to a table.
    Use clear separators like | or || between columns.
    
    Example format:
    Rank 1 || Plan Name || Insurer || Key Features: Feature1, Feature2 || Why: Reason1, Reason2 || Considerations: Note1
    """,
    agent=comparison_agent,
    expected_output="Structured comparison table of top 5 plans with detailed reasoning",
    context=[profile_task, recommend_task]
)

print("✅ Created 3 tasks:")
print("   1. User Profiling Task")
print("   2. Plan Recommendation Task")
print("   3. Comparison Generation Task")

In [ ]:
# Create and Run the Crew
print("\n" + "="*60)
print("STARTING HEALTH INSURANCE RECOMMENDATION ANALYSIS")
print("="*60)
print("\nThis may take 2-3 minutes as agents analyze all plans...\n")

crew = Crew(
    agents=[user_profiler, recommendation_agent, comparison_agent],
    tasks=[profile_task, recommend_task, compare_task],
    process=Process.sequential,
    verbose=True
)

# Execute the crew
result = crew.kickoff()

print("\n" + "="*60)
print("ANALYSIS COMPLETE!")
print("="*60)

In [ ]:
# Display the raw result
print("\n📊 RECOMMENDATION RESULTS:\n")
print(result)

In [ ]:
# Parse and format results into a DataFrame
# Note: The exact parsing will depend on the LLM output format
# This is a basic parser - you may need to adjust based on actual output

def parse_recommendations(result_text):
    """Parse the crew result into a structured format"""
    # This is a simplified parser - adjust based on actual output format
    lines = str(result_text).split('\n')
    
    recommendations = []
    
    # Look for structured data in the output
    for line in lines:
        if '||' in line:  # Our suggested separator
            parts = [p.strip() for p in line.split('||')]
            if len(parts) >= 4:
                rec = {
                    'Rank': parts[0],
                    'Plan Name': parts[1],
                    'Insurer': parts[2],
                    'Key Features': parts[3] if len(parts) > 3 else '',
                    'Why Recommended': parts[4] if len(parts) > 4 else '',
                    'Considerations': parts[5] if len(parts) > 5 else ''
                }
                recommendations.append(rec)
    
    if not recommendations:
        # Fallback: create a simple display of the text result
        print("\n⚠️  Could not parse into structured table. Showing raw results above.")
        print("\nTIP: The recommendations are in the output above. For better formatting,")
        print("you might need to adjust the parsing logic based on the LLM output format.")
        return None
    
    return pd.DataFrame(recommendations)

# Try to parse and display as DataFrame
df = parse_recommendations(result)

if df is not None and not df.empty:
    print("\n" + "="*80)
    print("RECOMMENDED HEALTH INSURANCE PLANS")
    print("="*80)
    print(df.to_string(index=False))
    print("\n✅ Recommendation complete!")
else:
    print("\n✅ Recommendation complete! See results above.")

## Demo Examples

Below are some pre-configured examples you can try:

In [ ]:
# Example User Profiles for Testing

demo_profiles = {
    'young_professional': {
        'age': '28',
        'pre_existing_conditions': 'None',
        'budget': '10000-15000',
        'specific_needs': 'No room rent limit, good claim settlement',
        'preferences': 'Wellness rewards, digital experience'
    },
    'family_with_kids': {
        'age': '35',
        'pre_existing_conditions': 'None',
        'budget': '20000-30000',
        'specific_needs': 'Maternity coverage, family floater, OPD',
        'preferences': 'Large hospital network, quick claim settlement'
    },
    'senior_citizen': {
        'age': '62',
        'pre_existing_conditions': 'Diabetes, Hypertension',
        'budget': '30000-50000',
        'specific_needs': 'Short PED waiting, no co-payment, home care',
        'preferences': 'Premium features, excellent CSR, restoration benefit'
    }
}

print("Demo Profiles Available:")
for name, profile in demo_profiles.items():
    print(f"\n{name.replace('_', ' ').title()}:")
    for key, value in profile.items():
        print(f"  - {key.replace('_', ' ').title()}: {value}")

print("\n\nTo use a demo profile, copy the values to the input cells above.")